In [4]:
import numpy as np
import pandas as pd
import time

from functools import partial
from scipy.stats import randint, uniform, beta
from sim_lib import simulation

pd.options.mode.chained_assignment = None


# Реализация policy
Полиси основано на алгоритме Thompson Sampling, подробности в комментариях:

In [17]:
class ThompsonSampler:
    
    def __init__(self):
        self.chosen_index = None    # айди баннера, показанного в прошлый раз
        self.clicks_of_chosen = None    # сколько кликов было у баннера до того, как мы его показали
        self.old_log = None # лог на момент конца предыдущей итерации

    def __call__(self, new_log):
        if self.old_log is None:    # кейс, когда мы первый раз вызываем полиси с первоначальным логом
            # всем баннерам проставляем параметры бета распределения, соответствующие равномерному: альфу и бету = 1
            new_log[["alpha", "beta"]] = 1
            # для каждого баннера из бета распределения семплируем тету
            new_log["theta"] = new_log[["alpha", "beta"]].apply(lambda x: beta.rvs(x[0], x[1]), axis=1)
            # берем аргмакс и получаем айди баннера с наибольшей тетой
            self.chosen_index = int(new_log[["theta"]].idxmax())
            # запоминаем, сколько у выбранного баннера было кликов
            self.clicks_of_chosen = new_log.loc[self.chosen_index, "clicks"]
            # запоминаем лог со всеми параметрами
            self.old_log = new_log 
            # возвращаем айди "лучшего" баннера
            return self.chosen_index    

        else:   # кейс, когда мы вызываем полиси не впервые
            # получаем список столбцов с параметрами, которых нет в свежем логе
            cols_to_use = self.old_log.columns.difference(new_log.columns)  

            # мерджим старый и новый лог
            new_log = pd.merge(new_log, self.old_log[cols_to_use], left_index=True, right_index=True, how='left')   
            # так как мы ничего не знаем о новых айтемах, параметры бета распределения соответствуют равномерному
            new_log[["alpha", "beta"]] = new_log[["alpha", "beta"]].fillna(1)
            
            # апдейтим параметры распределения ручки, которую мы дернули в прошлый раз в зависимости от того, был ли клик
            try:    # смотрим, не исчез ли баннер
                new_log.loc[self.chosen_index]
            except KeyError:    # когда баннер, который мы показали, исчезает из лога в конце итерации симулятора
                pass
            else:   # когда баннер не исчез
                if new_log.loc[self.chosen_index, "clicks"] > self.clicks_of_chosen:    # и на него кликнули
                    new_log.loc[self.chosen_index, "alpha"] += 1    # повышаем счетчик успехов
                else:       # когда на баннер не кликнули
                    new_log.loc[self.chosen_index, "beta"] += 1     # повышаем счетчик провалов
            
            # семплируем тету для каждого баннера
            new_log["theta"] = new_log[["alpha", "beta"]].apply(lambda x: beta.rvs(x[0], x[1]), axis=1)
            # выбираем баннер с наибольшей тетой
            self.chosen_index = int(new_log[["theta"]].idxmax())
            # запоминаем, сколько у него уже есть кликов
            self.clicks_of_chosen = new_log.loc[self.chosen_index, "clicks"]
            # запоминаем лог
            self.old_log = new_log

            # возвращаем "лучший" баннер
            return self.chosen_index

# Эксперимент

In [18]:
policy = ThompsonSampler()

In [19]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


1133.6219260692596

Выведем регрет:

In [20]:
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1471.230717361168, 0.00735615358680584, 179)

Итак, регрет при использовании нашей полиси намного ниже, чем при использовании бейзлайнового эпсилон-гриди бандита

In [21]:
output['history']

,impressions,clicks,lifetime,p
154,156.0,0.0,16595.715595,0.000442
169,3736.0,144.0,11415.767481,0.039232
175,4496.0,596.0,11381.152627,0.137817
178,42.0,1.0,4959.290645,0.030834


Ради интереса посмотрим, насколько наш эстимейт тета близок к реальной вероятности p, которую мы пытались приблизить.

In [22]:
policy.old_log

,impressions,clicks,alpha,beta,theta
154,156.0,0.0,1.0,157.0,0.008628
169,3736.0,144.0,145.0,3593.0,0.038500
175,4495.0,596.0,597.0,3900.0,0.132384
178,42.0,1.0,2.0,42.0,0.036934


In [23]:
output['history']['p'] - policy.old_log['theta']

154   -0.008185
169    0.000732
175    0.005433
178   -0.006100
dtype: float64

Как видим, отклонения от истинной вероятности для 4 последних живых баннеров не больше 0.0082, что довольно неплохо

# Воспроизведение бейзлайна
Чтобы все было честно, воспроизведем бейзлайновое решение

In [13]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.08)

# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


404.7350628376007

In [14]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(2792.237649427154, 0.01396118824713577, 174)

Как видим, регрет действительно выше, чем у нашего Томпсон семплера

In [15]:
output['history']

,impressions,clicks,lifetime,p
132,19843.0,1332.0,10870.812904,0.064972
162,154.0,2.0,18594.827945,0.017514
163,86.0,0.0,5153.010937,0.000849
164,68.0,2.0,5092.571727,0.041281
166,48.0,0.0,5340.552070,0.007253
167,33.0,0.0,1474.181162,0.033849
168,23.0,0.0,4900.260295,0.027273
169,14.0,0.0,5007.022458,0.030857
170,18.0,0.0,8920.324215,0.034653
171,14.0,0.0,1080.025985,0.012549
